In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# -------------------------
# 1. 資料預處理 (Data Augmentation)
# -------------------------
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),   # 隨機裁切 (增強泛化能力)
    transforms.RandomHorizontalFlip(),      # 隨機水平翻轉
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))  # 正規化 (CIFAR-10 常用均值/方差)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

In [3]:
# -------------------------
# 2. 模型 (ResNet18)
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = torchvision.models.resnet18(num_classes=10)  # CIFAR-10 10 類
model = model.to(device)


Using device: cuda


In [4]:
# -------------------------
# 3. Loss, Optimizer, Scheduler
# -------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)  # 200 epochs


In [5]:
# -------------------------
# 4. 訓練 & 測試函數
# -------------------------
def train(epoch):
    model.train()
    total, correct, running_loss = 0, 0, 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    acc = 100.*correct/total
    print(f"Epoch [{epoch}] Train Loss: {running_loss/len(train_loader):.3f}, Acc: {acc:.2f}%")

def test(epoch):
    model.eval()
    total, correct, test_loss = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    acc = 100.*correct/total
    print(f"Epoch [{epoch}] Test Loss: {test_loss/len(test_loader):.3f}, Acc: {acc:.2f}%")
    return acc

In [ ]:
# -------------------------
# 5. 訓練迴圈
# -------------------------
best_acc = 0
num_epochs = 200  # 建議 50~200，看時間長度

for epoch in range(1, num_epochs+1):
    train(epoch)
    acc = test(epoch)
    scheduler.step()

    # 儲存最佳模型
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_resnet18.pth")

print("Training finished. Best Test Accuracy: {:.2f}%".format(best_acc))

'''
Epoch [200] Train Loss: 0.259, Acc: 90.99%
Epoch [200] Test Loss: 0.530, Acc: 83.60%
Training finished. Best Test Accuracy: 89.19%
'''

Epoch [1] Train Loss: 0.615, Acc: 78.98%
Epoch [1] Test Loss: 0.808, Acc: 73.20%
Epoch [2] Train Loss: 0.623, Acc: 78.78%
Epoch [2] Test Loss: 0.722, Acc: 76.42%
Epoch [3] Train Loss: 0.611, Acc: 79.02%
Epoch [3] Test Loss: 0.833, Acc: 72.57%
Epoch [4] Train Loss: 0.609, Acc: 79.13%
Epoch [4] Test Loss: 0.922, Acc: 70.69%
Epoch [5] Train Loss: 0.614, Acc: 78.92%
Epoch [5] Test Loss: 0.705, Acc: 76.29%
Epoch [6] Train Loss: 0.604, Acc: 79.52%
Epoch [6] Test Loss: 0.731, Acc: 76.18%
Epoch [7] Train Loss: 0.606, Acc: 79.32%
Epoch [7] Test Loss: 0.631, Acc: 78.52%
Epoch [8] Train Loss: 0.603, Acc: 79.43%
Epoch [8] Test Loss: 0.756, Acc: 74.31%
Epoch [9] Train Loss: 0.601, Acc: 79.27%
Epoch [9] Test Loss: 0.675, Acc: 77.36%
Epoch [10] Train Loss: 0.596, Acc: 79.57%
Epoch [10] Test Loss: 0.625, Acc: 78.73%
Epoch [11] Train Loss: 0.597, Acc: 79.65%
Epoch [11] Test Loss: 0.685, Acc: 76.72%
Epoch [12] Train Loss: 0.595, Acc: 79.67%
Epoch [12] Test Loss: 0.687, Acc: 77.05%
Epoch [13] Train Loss: